In [ ]:
!whoami
!pwd
!pip show sagemaker


In [ ]:
import sys

# 1) Quitar la v3 (y submódulos nuevos)
#!{sys.executable} -m pip uninstall -y sagemaker sagemaker-core sagemaker-train sagemaker-serve sagemaker-mlops

# 2) Instalar la rama 2.x (cualquier versión < 3.0.0)
#!{sys.executable} -m pip install "sagemaker<3.0.0"


In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker.sklearn.model import SKLearnModel

print("Versión de sagemaker:", sagemaker.__version__)

# Project Description

The main objective of this project is to deploy different machine learning models in SageMaker and interact with them through a web page. The models are trained in different notebooks and then deployed in SageMaker using the `deploy.ipynb` notebook. The web page is a React application that allows the user to interact with the deployed models.

# Project Architecture

The project is structured in three main parts: the models, the web page and the deployment. The models are in the `modelos` folder, the web page in the `page` folder and the deployment in the `deploy.ipynb` file.

The `modelos` folder contains the models that are going to be deployed. Each model has its own folder with the necessary files to deploy it. The `code` folder contains the `inference.py` file with the functions required by SageMaker to deploy the model. The `model.joblib` and `vectorizer.joblib` files are the model and the vectorizer that are going to be deployed.

The `page` folder contains the web page that is going to be used to interact with the deployed models. The web page is a React application that is going to be deployed in a S3 bucket.

The `deploy.ipynb` file contains the code to deploy the models in SageMaker. The notebook is divided in three parts: the creation of the `tar.gz` files, the upload of the models to S3 and the deployment of the models in SageMaker.


# Create tar.gz files

## Neumonia

```bash
tar -czvf modelos/neumonia/model.tar.gz -C modelos/neumonia/ model.joblib -C modelos/neumonia/code/ .
```

## SVM Count Vectorizer

```bash
tar -czvf modelos/sentimientos/svm_countvectorizer/model.tar.gz -C modelos/sentimientos/svm_countvectorizer/ model.joblib vectorizer.joblib -C modelos/sentimientos/svm_countvectorizer/code/ .
```

## SVM TF-IDF Vectorizer

```bash
tar -czvf modelos/sentimientos/svm_tfidfvectorizer/model.tar.gz -C modelos/sentimientos/svm_tfidfvectorizer/ model.joblib vectorizer.joblib -C modelos/sentimientos/svm_tfidfvectorizer/code/ .
```

In [ ]:
import boto3
s3 = boto3.client('s3')

# Upload Neumonia model
s3.upload_file(
    Filename="./modelos/neumonia/model.tar.gz",
    Bucket='modelos-sagemaker',
    Key='modelos/neumonia/model.tar.gz'
)

# Upload SVM Count Vectorizer model
s3.upload_file(
    Filename="./modelos/sentimientos/svm_countvectorizer/model.tar.gz",
    Bucket='modelos-sagemaker',
    Key='modelos/sentimientos/svm_countvectorizer/model.tar.gz'
)

# Upload SVM TF-IDF Vectorizer model
# s3.upload_file(
#     Filename="./modelos/sentimientos/svm_tfidfvectorizer/model.tar.gz",
#     Bucket='modelos-sagemaker',
#     Key='modelos/sentimientos/svm_tfidfvectorizer/model.tar.gz'
# )

: 

# Deploy Neumonia Model

In [ ]:
role = sagemaker.get_execution_role()

neumonia_model = SKLearnModel(
    name='neumonia-model',
    model_data='s3://modelos-sagemaker/modelos/neumonia/model.tar.gz',
    role=role,
    entry_point='inference.py',
    source_dir='modelos/neumonia/code',
    framework_version='1.0-1',
    py_version='py3'
)

neumonia_predictor = neumonia_model.deploy(
    endpoint_name='neumonia-endpoint',
    instance_type='ml.m5.large',
    initial_instance_count=1
)

# Deploy SVM Count Vectorizer Model

In [ ]:
role = sagemaker.get_execution_role()

svm_countvectorizer_model = SKLearnModel(
    name='svm-countvectorizer-model',
    model_data='s3://modelos-sagemaker/modelos/sentimientos/svm_countvectorizer/model.tar.gz',
    role=role,
    entry_point='inference.py',
    source_dir='modelos/sentimientos/svm_countvectorizer/code',
    framework_version='1.0-1',
    py_version='py3'
)

svm_countvectorizer_predictor = svm_countvectorizer_model.deploy(
    endpoint_name='svm-countvectorizer-endpoint',
    instance_type='ml.m5.large',
    initial_instance_count=1
)

# Deploy SVM TF-IDF Vectorizer Model (Commented Out)

In [ ]:
# role = sagemaker.get_execution_role()
#
# svm_tfidfvectorizer_model = SKLearnModel (
#     model_data='s3://modelos-sagemaker/modelos/sentimientos/svm_tfidfvectorizer/model.tar.gz',
#     role=role,
#     entry_point='inference.py',
#     source_dir='modelos/sentimientos/svm_tfidfvectorizer/code',
#     framework_version='1.0-1',
#     py_version='py3'
# )
#
# svm_tfidfvectorizer_predictor = svm_tfidfvectorizer_model.deploy (
#     endpoint_name='svm-tfidfvectorizer-endpoint',
#     instance_type='ml.m5.large',
#     initial_instance_count=1
# )

# Gestión y Actualización de Modelos en SageMaker

Una vez que un modelo está desplegado en un endpoint, es común querer actualizarlo con una nueva versión (por ejemplo, re-entrenado con más datos o mejores hiperparámetros) sin interrumpir el servicio.

## ¿Cómo actualizar un modelo sin afectar el endpoint?

El proceso general es el siguiente:

1.  **Preparar el nuevo artefacto del modelo**: Vuelve a entrenar tu modelo y empaqueta los nuevos archivos (`model.joblib`, `vectorizer.joblib`, etc.) en un nuevo archivo `model.tar.gz`.

2.  **Subir el nuevo artefacto a S3**: Sube el nuevo `model.tar.gz` a S3. Es una buena práctica usar un versionado en la ruta de S3 para no sobreescribir los modelos antiguos. Por ejemplo:
    ```python
    # Ejemplo de nueva ruta en S3
    new_model_data = 's3://modelos-sagemaker/modelos/neumonia/v2/model.tar.gz'
    ```

3.  **Crear un nuevo 'Model Object' en SageMaker**: Usando el SDK de SageMaker, crea una nueva instancia de `SKLearnModel` (o el tipo de modelo que corresponda) apuntando a la nueva ruta de S3. Es crucial darle un **nuevo nombre de modelo** para no sobreescribir la definición del modelo anterior.
    ```python
    from sagemaker.sklearn.model import SKLearnModel

    new_neumonia_model = SKLearnModel (
        name='neumonia-model-v2',  # Nuevo nombre para el modelo
        model_data=new_model_data, # Ruta al nuevo tar.gz
        role=role,
        entry_point='inference.py',
        source_dir='modelos/neumonia/code',
        framework_version='1.0-1',
        py_version='py3'
    )
    ```

4.  **Actualizar el Endpoint**: Llama al método `deploy()` desde el nuevo objeto de modelo, pero especificando el **mismo nombre de endpoint** que ya existe. SageMaker se encargará de crear una nueva configuración de endpoint y redirigir el tráfico al nuevo modelo de forma segura (blue/green deployment).
    ```python
    # El endpoint 'neumonia-endpoint' ya existe, SageMaker lo actualizará
    new_neumonia_model.deploy (
        endpoint_name='neumonia-endpoint', 
        instance_type='ml.m5.large',
        initial_instance_count=1
    )
    ```
    SageMaker realizará el despliegue de forma automática, y una vez que el nuevo modelo esté operativo, dará de baja la versión anterior sin tiempo de inactividad.

## ¿Cómo se gestiona la metadata del modelo?

La "metadata" de un modelo en SageMaker se gestiona a través de varios componentes:

-   **Nombres de Modelo (`name`)**: Como se mostró, al crear un `SKLearnModel` o similar, puedes asignarle un nombre único. Este nombre aparece en la consola de SageMaker en "Modelos" y es la referencia principal al artefacto y su configuración.

-   **Tags (Etiquetas)**: Tanto a los modelos como a los endpoints se les pueden añadir "tags", que son pares clave-valor. Son muy útiles para organizar, filtrar y gestionar costos. Por ejemplo, puedes añadir tags como `version`, `proyecto`, `ambiente`, etc.
    ```python
    # Ejemplo de cómo añadir tags al desplegar
    neumonia_predictor = neumonia_model.deploy(
        endpoint_name='neumonia-endpoint',
        instance_type='ml.m5.large',
        initial_instance_count=1,
        tags=[
            {'Key': 'Project', 'Value': 'AnalisisMedico'},
            {'Key': 'Version', 'Value': '1.0'}
        ]
    )
    ```

-   **Descripción del Modelo**: Al crear el modelo a través de la consola o la API de boto3, puedes añadir una descripción más detallada. Con el SDK de alto nivel, la descripción no es un parámetro directo, pero las tags pueden suplir esta función.

-   **SageMaker Model Registry**: Para una gestión más avanzada y formal del ciclo de vida de los modelos, existe SageMaker Model Registry. Permite versionar modelos, aprobarlos o rechazarlos, y asociarles métricas de evaluación. Es ideal para entornos de producción serios donde la trazabilidad y la gobernanza son importantes.